In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,cross_val_score

In [ ]:
df=pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")

In [ ]:
df.isna().sum()

In [ ]:
impute=IterativeImputer(estimator=LogisticRegression())
df.loc[:,['bmi']]=impute.fit_transform(df.loc[:,['bmi']])

In [ ]:
df.isna().sum()

In [ ]:
X=df.iloc[:,:-1]

In [ ]:
y=df.iloc[:,1]

In [ ]:
cat_fea=[feature for feature in X.columns if X[feature].dtype=='O']

In [ ]:
categorical_feature=X[cat_fea]

In [ ]:
categorical_feature.head()

In [ ]:
categorical_feature=pd.get_dummies(categorical_feature,drop_first=True)

In [ ]:
num_fea=[feature for feature in X.columns if X[feature].dtype!='O']

In [ ]:
numerical_feature=X[num_fea]

In [ ]:
numerical_feature.head()

In [ ]:
numerical_feature.drop('id',axis=1,inplace=True)

In [ ]:
numerical_feature.head()

In [ ]:
final_data=pd.concat([numerical_feature,categorical_feature],axis=1)

In [ ]:
y=df.iloc[:,-1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_data, y, test_size=0.3, random_state=101)

In [ ]:
st=StandardScaler()
pc=PCA()
lm=LogisticRegression()
rf=RandomForestClassifier()
sm=SMOTE()
xgboost=XGBClassifier()

In [ ]:
logistic_pipe=make_pipeline(sm,st,pc,lm)

In [ ]:
rf_pipe=make_pipeline(sm,rf)

In [ ]:
pipeline=[logistic_pipe,rf_pipe]
for i in pipeline:
    i.fit(X_train,y_train)

In [ ]:
dic={0:'Logistic Regression',1:'Random Forest'}

In [ ]:
for i,j in enumerate(pipeline):
    print("{} Test Accuracy {}".format(dic[i],cross_val_score(j,X_test,y_test,cv=10).mean()))

In [ ]:
for i,j in enumerate(pipeline):
    print("{} Train Accuracy {}".format(dic[i],cross_val_score(j,X_train,y_train,cv=10).mean()))